# BentoML Demo: Iris Classifier with custom web UI


BentoML is an open-source framework for machine learning **model serving**, aiming to **bridge the gap between Data Science and DevOps.**

Data Scientists can easily package their models trained with any ML framework using BentoMl and reproduce the model for serving in production. BentoML helps with managing packaged models in the BentoML format, and allows DevOps to deploy them as online API serving endpoints or offline batch inference jobs, on any cloud platform.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.


This notebook demonstrates how to use BentoML to __serve a Iris Classification model containing a REST API server with Custom Web UI__.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=scikit-learn&ea=scikit-learn-iris-classifier-web&dt=scikit-learn-iris-classifier-web)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install -q bentoml "scikit-learn>=0.23.2"

## Create BentoService for model serving

In [1]:
%%writefile iris_classifier.py
from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import DataframeInput
from bentoml.artifact import SklearnModelArtifact

@env(auto_pip_dependencies=True)
@artifacts([SklearnModelArtifact('model')])
class IrisClassifier(BentoService):

    @api(input=DataframeInput())
    def predict(self, df):
        # Optional pre-processing, post-processing code goes here
        return self.artifacts.model.predict(df)

Writing iris_classifier.py


In [2]:
%%writefile main.py
from sklearn import svm
from sklearn import datasets

from iris_classifier import IrisClassifier

if __name__ == "__main__":
    # Load training data
    iris = datasets.load_iris()
    X, y = iris.data, iris.target

    # Model Training
    clf = svm.SVC(gamma='scale')
    clf.fit(X, y)

    # Create a iris classifier service instance
    iris_classifier_service = IrisClassifier()

    # Pack the newly trained model artifact
    iris_classifier_service.pack('model', clf)

    # Save the prediction service to disk for model serving
    saved_path = iris_classifier_service.save()

Writing main.py


In [3]:
!python main.py

[2020-09-09 11:50:39,692] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-09 11:50:41,005] INFO - Detected non-PyPI-released BentoML installed, copying local BentoML modulefiles to target saved bundle path..
no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'
UPDATING BentoML-0.8.6+34.g6123b8c6/bentoml/_version.py
set BentoML-0.8.6+34.g6123b8c6/bentoml/_version.py to '0.8.6+34.g6123b8c6'
[2020-09-09 11:50:45,682] INFO - BentoService bundle 'IrisClassifier:20200909115040_9FC7F4' saved to: /Users/bozhaoyu/bentoml/repository/IrisClassifier/20200909115040_9FC7F4


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [6]:
!bentoml serve IrisClassifier:latest

[2020-07-28 16:38:55,465] INFO - Getting latest version IrisClassifier:20200728163753_DE85E9
[2020-07-28 16:38:56,409] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-07-28 16:38:56,422] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.1, but loading from BentoML version 0.8.1+27.g8e155f5.dirty
 * Serving Flask app "IrisClassifier" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
^C


If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [ ]:
!bentoml serveIrisiClassifier:latest --run-with-ngrok

At this point you can test out the Rest API server either by opening http://localhost:5000 in a new tab which will serve the swagger docs:

![alt text](https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/swagger.png)


or by making a curl request in a another terminal window:

```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '[[1,2,1,2]]' \
localhost:5000/predict
```

## Adding Custom Web Static Content


In [7]:
!curl https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/static.tar.xz -o static.tar.xz
!tar --xz -xf static.tar.xz
!rm static.tar.xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  197k  100  197k    0     0   200k      0 --:--:-- --:--:-- --:--:--  200k


Here we have a very simple web ui as our static content that bento will serve.
Now we will edit our bento service to point to this static directory.

Add `@web_static_content('./static')` to `iris_classifier.py`

**Note**: The path can be both relative or absolute. 

In [9]:
%%writefile iris_classifier.py
from bentoml import env, artifacts, api, BentoService, web_static_content
from bentoml.adapters import DataframeInput
from bentoml.artifact import SklearnModelArtifact

@env(auto_pip_dependencies=True)
@artifacts([SklearnModelArtifact('model')])
@web_static_content('./static')
class IrisClassifier(BentoService):

    @api(input=DataframeInput())
    def test(self, df):
        # Optional pre-processing, post-processing code goes here
        return self.artifacts.model.predict(df)

Overwriting iris_classifier.py


In [10]:
!python main.py

[2020-07-30 09:09:28,868] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-07-30 09:09:39,344] INFO - Detect BentoML installed in development model, copying local BentoML module file to target saved bundle path
running sdist
running egg_info
writing BentoML.egg-info/PKG-INFO
writing dependency_links to BentoML.egg-info/dependency_links.txt
writing entry points to BentoML.egg-info/entry_points.txt
writing requirements to BentoML.egg-info/requires.txt
writing top-level names to BentoML.egg-info/top_level.txt
reading manifest file 'BentoML.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'
writing manifest file 

In [ ]:
!bentoml serve IrisClassifier:latest

[2020-07-30 09:09:43,808] INFO - Getting latest version IrisClassifier:20200730090929_C02FB7
[2020-07-30 09:09:43,808] INFO - Starting BentoML API server in development mode..
[2020-07-30 09:09:44,916] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-07-30 09:09:44,949] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.3, but loading from BentoML version 0.8.3+42.gb8d36b6
 * Serving Flask app "IrisClassifier" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jul/2020 09:09:52] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2020 09:09:55] "POST /test HTTP/1.1" 200 -
127.0.0.1 - - [30/Jul/2020 09:09:57] "PO

Now if you visit http://localhost:5000/, you should be served with a beautiful UI:

![Custom UI](https://raw.githubusercontent.com/bentoml/gallery/master/scikit-learn/iris-classifier/webui.png)

It's still possible to access the swagger docs at `/docs`

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **note available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [4]:
!bentoml containerize IrisClassifier:latest

[2020-09-09 11:52:05,072] INFO - Getting latest version IrisClassifier:20200909115040_9FC7F4
Found Bento: /Users/bozhaoyu/bentoml/repository/IrisClassifier/20200909115040_9FC7F4
[2020-09-09 11:52:05,109] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-09 11:52:05,126] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.8.6, but loading from BentoML version 0.8.6+34.g6123b8c6
Tag not specified, using tag parsed from BentoService: 'irisclassifier:20200909115040_9FC7F4'
Building Docker image irisclassifier:20200909115040_9FC7F4 from IrisClassifier:latest 
-we in here
processed docker file
(None, None)
root in create archive /Users/bozhaoyu/bentoml/repository/IrisClassifier/20200909115040_9FC7F4 ['Dockerfile', 'IrisClassifier', 'IrisClassi

/Requirement already satisfied: prometheus-client in /opt/conda/lib/python3.7/site-packages (from bentoml==0.8.6->-r ./requirements.txt (line 3)) (0.8.0)
|Requirement already satisfied: alembic in /opt/conda/lib/python3.7/site-packages (from bentoml==0.8.6->-r ./requirements.txt (line 3)) (1.4.2)
\Requirement already satisfied: humanfriendly in /opt/conda/lib/python3.7/site-packages (from bentoml==0.8.6->-r ./requirements.txt (line 3)) (8.2)
-Requirement already satisfied: thriftpy2>=0.4.0 in /opt/conda/lib/python3.7/site-packages (from py-zipkin->bentoml==0.8.6->-r ./requirements.txt (line 3)) (0.4.11)
/Installing collected packages: threadpoolctl, joblib, scipy, scikit-learn, pytz, pandas
-Successfully installed joblib-0.16.0 pandas-1.1.2 pytz-2020.1 scikit-learn-0.23.2 scipy-1.5.2 threadpoolctl-2.1.0
-+ for filename in ./bundled_pip_dependencies/*.tar.gz
+ '[' -e ./bundled_pip_dependencies/BentoML-0.8.6+34.g6123b8c6.tar.gz ']'
+ pip install -U ./bundled_pip_dependencies/Bento

/Requirement already satisfied, skipping upgrade: numpy in /opt/conda/lib/python3.7/site-packages (from BentoML==0.8.6+34.g6123b8c6) (1.19.1)
|Collecting sqlalchemy-utils<0.36.8
-  Downloading SQLAlchemy-Utils-0.36.7.tar.gz (131 kB)
|Requirement already satisfied, skipping upgrade: ruamel.yaml>=0.15.0 in /opt/conda/lib/python3.7/site-packages (from BentoML==0.8.6+34.g6123b8c6) (0.15.87)
\Requirement already satisfied, skipping upgrade: humanfriendly in /opt/conda/lib/python3.7/site-packages (from BentoML==0.8.6+34.g6123b8c6) (8.2)
-Requirement already satisfied, skipping upgrade: Mako in /opt/conda/lib/python3.7/site-packages (from alembic->BentoML==0.8.6+34.g6123b8c6) (1.1.3)
\  Building wheel for BentoML (PEP 517): finished with status 'done'
-  Created wheel for BentoML: filename=BentoML-0.8.6+34.g6123b8c6-py3-none-any.whl size=4712455 sha256=e6944330cc4d24e21e20865bf3b08f780a9c2fde3fbc129cbda87c7f36aef89b
  Stored in directory: /tmp/pip-ephem-wheel-cache-n87zjgsc/wheels/be/

-  Building wheel for sqlalchemy-utils (setup.py): finished with status 'done'
  Created wheel for sqlalchemy-utils: filename=SQLAlchemy_Utils-0.36.7-py2.py3-none-any.whl size=93226 sha256=2f7ba4b0a4e6875f0b834144d9850d2a7e7e952d19eb97b5225e8fe8849415cb
  Stored in directory: /tmp/pip-ephem-wheel-cache-n87zjgsc/wheels/85/a9/d2/3377194742a9ad5eb57ee36f934438c15e1aaa0843dd41c899
Successfully built BentoML sqlalchemy-utils
|Installing collected packages: sqlalchemy-utils, BentoML
  Attempting uninstall: sqlalchemy-utils
    Found existing installation: SQLAlchemy-Utils 0.36.8
    Uninstalling SQLAlchemy-Utils-0.36.8:
\      Successfully uninstalled SQLAlchemy-Utils-0.36.8
-  Attempting uninstall: BentoML
    Found existing installation: BentoML 0.8.6
/    Uninstalling BentoML-0.8.6:
\      Successfully uninstalled BentoML-0.8.6
/Successfully installed BentoML-0.8.6+34.g6123b8c6 sqlalchemy-utils-0.36.7
| ---> 0048faff09db
Step 10/15 : ENV PORT 5000
\ ---> Running in 1d42a7a4b733


In [ ]:
!docker run --rm -p 5000:5000 irisclassifier:20200909115040_9FC7F4 

## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [ ]:
!bentoml run IrisClassifier:latest predict --input '[[5, 4, 3, 2]]'

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [ ]:
from bentoml import load

service = load(saved_path)

print(service.predict([[5,4,3,2]]))

# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)

